# Simple Generative App Using Palm 2 & LangChain

## Installation & Authentication

**Install google-generativeai & langchain**
- Get API KEY from MakerSuite or Google Cloud

In [9]:
GOOGLE_API_KEY='APIKey'

In [ ]:
!pip install langchain google-generativeai

**(Or) Authenticate & Initialize Google Cloud Project**

In [6]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

PROJECT_ID = "api-project-503433767370"  # @param {type:"string"}

LOCATION = "us-central1"  # @param {type:"string"}


**Install Google Cloud AI Platform and Define Vertex AI Base Model & Class**

In [ ]:
# Install Vertex AI LLM SDK# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform==1.25.0

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")


from pydantic import BaseModel, root_validator
from typing import Any, Mapping, Optional, List, Dict
from langchain.llms.base import LLM

class _VertexCommon(BaseModel):
    """Wrapper around Vertex AI large language models.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    client: Any = None #: :meta private:
    model_name: str = "text-bison@001"
    """Model name to use."""

    temperature: float = 0.2
    """What sampling temperature to use."""

    top_p: int = 0.8
    """Total probability mass of tokens to consider at each step."""

    top_k: int = 40
    """The number of highest probability tokens to keep for top-k filtering."""

    max_output_tokens: int = 200
    """The maximum number of tokens to generate in the completion."""

    @property
    def _default_params(self) -> Mapping[str, Any]:
        """Get the default parameters for calling Vertex AI API."""
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vertex_ai"

class VertexLLM(_VertexCommon, LLM):
    model_name: str = "text-bison@001"

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to Vertex AI's create endpoint.

        Args:
            prompt: The prompt to pass into the model.

        Returns:
            The string generated by the model.
        """
        return self._predict(prompt, stop)



## Initiatlize Vertex AI

In [22]:
# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Validation & Helper Functions

In [7]:
import requests

# Get the access token.
gcloud_token = !gcloud auth print-access-token

# Get the token info.
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()

# Print the user ID.
print(gcloud_tokeninfo['sub'])
# Print the user email.
print(gcloud_tokeninfo['email'])


110961358251290988711
guruprakash.cr@gmail.com


In [43]:
from langchain import chat_models
from langchain.chat_models import ChatGooglePalm
from vertexai.preview.language_models import ChatModel, InputOutputTextPair


#print("Help of chat_models is:", help(ChatGooglePalm))
print("Dir of chat_models is:", dir(chat_models))
print("ID of chat_models is:", id(chat_models))
print("Type of chat_models is:", type(chat_models))
print("Dir of ChatGooglePalm is:", dir(ChatGooglePalm))

Dir of chat_models is: ['AzureChatOpenAI', 'BedrockChat', 'ChatAnthropic', 'ChatAnyscale', 'ChatCohere', 'ChatFireworks', 'ChatGooglePalm', 'ChatJavelinAIGateway', 'ChatKonko', 'ChatLiteLLM', 'ChatMLflowAIGateway', 'ChatOllama', 'ChatOpenAI', 'ChatVertexAI', 'ErnieBotChat', 'FakeListChatModel', 'HumanInputChatModel', 'JinaChat', 'MiniMaxChat', 'PromptLayerChatOpenAI', 'QianfanChatEndpoint', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'anthropic', 'anyscale', 'azure_openai', 'baidu_qianfan_endpoint', 'base', 'bedrock', 'cohere', 'ernie', 'fake', 'fireworks', 'google_palm', 'human', 'javelin_ai_gateway', 'jinachat', 'konko', 'litellm', 'minimax', 'mlflow_ai_gateway', 'ollama', 'openai', 'promptlayer_openai', 'vertexai']
ID of chat_models is: 133061509844000
Type of chat_models is: <class 'module'>
Dir of ChatGooglePalm is: ['Config', 'InputType', 'OutputType', '__abstractmethods__', '__annotations__', '_

##Invoking Chat Bison Without LangChain Prompt Template

In [48]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

chat = chat_model.start_chat(
        context="My name is Miles. You are an astronomer, knowledgeable about the solar system.",
        examples=[
            InputOutputTextPair(
                input_text="How many moons does Mars have?",
                output_text="The planet Mars has two moons, Phobos and Deimos.",
            ),
        ],
    )
parameters = {
        "temperature": 0.2,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
}
response = chat.send_message(
        "i dont like eating tasty things.", **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: I'm sorry to hear that. Tasty things are one of the best parts of life!


##Invoking Chat Bison Using LangChain Prompt Template

*Building a Prompt Template with LangChain*

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
print(template.format_messages(text='i dont like eating tasty things.'))

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')]


*Invoke Chat Bison*

In [49]:
llm = ChatGooglePalm(google_api_key=GOOGLE_API_KEY, temperature=0.5, top_k=30)
llm(template.format_messages(text='i dont like eating tasty things.'))

ChatMessage(content="That's okay! There are many other things to enjoy in life besides food. You could try spending time with friends and family, pursuing your hobbies, or learning something new. There's no need to force yourself to eat something you don't like, and there are plenty of other ways to have a good time.", role='1')

## Invoking Text Bison directly

In [41]:
llm = VertexLLM(
    model_name='text-bison-32k',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)


llm('i dont like eating tasty things.')
#llm(template.format_messages(text='i dont like eating tasty things.'))

' There are many reasons why someone might not like eating tasty things. Some people may have a medical condition that makes it difficult or painful to eat, while others may simply not enjoy the taste of food. Still others may have a psychological aversion to food, or may have had a negative experience with food in the past. If you are concerned about your lack of appetite, it is important to talk to your doctor to rule out any underlying medical conditions.'